In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

In [2]:
# 輸出資料的資料夾
out_folder_name = '新整理_旅館業住客類別及國籍人數統計表'
out_folder_path = os.path.join('final_report', out_folder_name)
os.makedirs(out_folder_path, exist_ok=True)

In [3]:
# 2020年以前
# 輸入資料的資料夾
fold_path = '/Users/liaoyayun/Documents/final_report/旅館業住客類別及國籍人數統計表_csv'
files_name = os.listdir(fold_path)

for file_name in files_name:
    if file_name.endswith('.csv'):
        # 組成檔案路徑
        file_path = os.path.join(fold_path, file_name)
        # 讀取csv
        data = pd.read_csv(file_path)
        
        # 填滿 row1
        data.iloc[1].fillna(method='ffill', inplace=True)
        
        # 合併 1、2row
        data.iloc[1,1:3] = data.iloc[1,1:3] + '_' + data.iloc[2,1:3]
        data.iloc[1,4:17] = data.iloc[1,4:17] + '_' + data.iloc[2,4:17]
        data.iloc[1,18:] = data.iloc[1,18:] + '_' + data.iloc[2,18:]
        
        # new column name
        new_column_name = data.iloc[1]
        data.columns = new_column_name
        
        # 新增column 日期
        regex01 = r'\d+年\d+月'
        string01 = data.iloc[0, 0]
        match = re.search(regex01, string01)

        if match:
            date_value = match.group()
            data['日期'] = date_value
            
        # 移除不要的row
        data = data.drop(data.index[[0,1,2,-1]])
        
        # 改日期格式
        date_str = str(data.iloc[0,-1])
        date_obj = datetime.strptime(date_str, "%Y年%m月")
        formatted_date = date_obj.strftime("%Y/%m/%d")
        data['日期'] = formatted_date
        
        # 日期column 移到最前面
        date_column = data['日期']
        data = data.drop('日期', axis=1)
        data.insert(0, '日期', date_column)
        
        # 改column名稱
        data.rename(columns={'縣市/項目': '縣市'}, inplace=True)
        data.rename(columns={'住宿國籍_大陸': '住宿國籍_中國大陸'}, inplace=True)
        
        # 需要删除的column
        columns_to_delete = ['住宿人數合計','接待外籍旅客家數(本國以外)', '員工人數_客房部', '員工人數_餐飲部', '員工人數_管理部', '員工人數_其他部門', '員工人數_合計']

        # 删除指定的column
        data = data.drop(columns=columns_to_delete)

        # 儲存到 新整理hotel
        out_file_name = file_name.replace('.csv', '_processed.csv')
        out_file_path = os.path.join(out_folder_path, out_file_name)
        data.to_csv(out_file_path, index=False)


In [16]:
# 2021年以後
# 輸出資料的資料夾
out_folder_path = '新整理_旅館業住客類別及國籍人數統計表'
# 輸入資料的資料夾
fold_path = '旅館業住客類別及國籍人數統計表_csv/2021-2023'

# 取得資料夾內所有檔案名稱
files_name = os.listdir(fold_path)

for file_name in files_name:
    if file_name.endswith('.csv'):
        # 組成檔案路徑
        file_path = os.path.join(fold_path, file_name)
        # 讀取csv檔
        data = pd.read_csv(file_path)
        
        # 刪除NaN的column
        data = data.drop(data.columns[1], axis=1)
        
        # 新的column名稱
        data.iloc[1,0] = '住客類別'
        data.iloc[1,3] = '住宿國籍'
        
        # 填滿 row1
        data.iloc[1].fillna(method='ffill', inplace=True)
        
        # 合併 1、2row
        data.iloc[2,0] = '縣市'
        data.iloc[2,1] = data.iloc[1,1] + '_' + '個別旅客'
        data.iloc[2,2] = data.iloc[1,2] + '_' + '團體'
        data.iloc[2,3:] = data.iloc[1,3:] + '_' + data.iloc[2,3:]
        
        # new column name
        new_column_name = data.iloc[2]
        data.columns = new_column_name
        
        # 新增column 日期
        regex01 = r'\d+年\d+月'
        string01 = data.iloc[0, 0]
        match = re.search(regex01, string01)

        if match:
            date_value = match.group()
            data['日期'] = date_value
            
        # 移除不要的row
        data = data.drop(data.index[[0,1,2,-1]])
        
        # 改日期格式
        date_str = str(data.iloc[0,-1])
        date_obj = datetime.strptime(date_str, "%Y年%m月")
        formatted_date = date_obj.strftime("%Y/%m/%d")
        data['日期'] = formatted_date
        
        # 日期column 移到最前面
        date_column = data['日期']
        data = data.drop('日期', axis=1)
        data.insert(0, '日期', date_column)
        
        # 新增 住宿國籍_其他亞洲地區、住宿國籍_歐洲、住宿國籍_北美
        data['住宿國籍_其他亞洲地區'] = data['住宿國籍_泰國'].astype(float) + data['住宿國籍_印尼'].astype(float) + data['住宿國籍_越南'].astype(float) + data['住宿國籍_菲律賓'].astype(float) + data['住宿國籍_汶萊'].astype(float) + data['住宿國籍_緬甸'].astype(float) + data['住宿國籍_寮國'].astype(float) + data['住宿國籍_柬埔寨'].astype(float) + data['住宿國籍_印度'].astype(float)
        data['住宿國籍_歐洲'] = data['住宿國籍_英國'].astype(float) + data['住宿國籍_歐洲其他地區'].astype(float)
        data['住宿國籍_北美'] = data['住宿國籍_美國'].astype(float) + data['住宿國籍_加拿大'].astype(float)

        
        # 改column名稱
        data.rename(columns={'住宿國籍_南韓': '住宿國籍_韓國'}, inplace=True)
        
        # 需要删除的column
        columns_to_delete = ['住宿國籍_英國', '住宿國籍_歐洲其他地區','住宿國籍_美國','住宿國籍_加拿大','住宿國籍_泰國','住宿國籍_印尼','住宿國籍_越南','住宿國籍_菲律賓','住宿國籍_汶萊','住宿國籍_緬甸','住宿國籍_寮國','住宿國籍_柬埔寨','住宿國籍_印度']

        # 删除指定的column
        data = data.drop(columns=columns_to_delete)
        
        # 儲存到 新整理hotel
        out_file_name = file_name.replace('.csv', '_processed.csv')
        out_file_path = os.path.join(out_folder_path, out_file_name)
        data.to_csv(out_file_path, index=False)